### Iphone

In [1]:
# pip install selenium webdriver-manager beautifulsoup4 lxml numpy pandas

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager

from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re, time, random


# ===================== Helpers =====================

def _to_int_digits(text):
    """Keep only digits and cast to int, else NaN."""
    if not text:
        return np.nan
    digits = re.sub(r"\D", "", text)
    return int(digits) if digits else np.nan


def _parse_ratings_reviews(text):
    """'25,319 Ratings & 1,097 Reviews' -> (25319, 1097)"""
    if not text:
        return np.nan, np.nan
    nums = [int(x.replace(",", "")) for x in re.findall(r"\d[\d,]*", text)]
    if len(nums) >= 2:
        return nums[0], nums[1]
    if len(nums) == 1:
        return nums[0], np.nan
    return np.nan, np.nan


def make_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless")
    drv = webdriver.Firefox(
        service=FirefoxService(GeckoDriverManager().install()),
        options=opts
    )
    drv.set_page_load_timeout(60)
    return drv


def get_page_soup(driver, url, max_scrolls=3, wait_css="div.KzDlHZ"):
    """Load URL, close popup, wait for product titles, scroll a bit, return BeautifulSoup or None."""
    for attempt in range(2):
        try:
            driver.get(url)

            # Close login popup if visible
            try:
                WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button._2KpZ6l._2doB4z"))
                ).click()
            except Exception:
                pass

            WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, wait_css))
            )

            # Light scroll to load lazy content
            last_h = 0
            for _ in range(max_scrolls):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.0)
                h = driver.execute_script("return document.body.scrollHeight;")
                if h == last_h:
                    break
                last_h = h

            return BeautifulSoup(driver.page_source, "lxml")

        except TimeoutException:
            if attempt == 0:
                time.sleep(2 + random.random() * 2)
                continue
            return None
    return None


# Regex for exchange phrase
UPTO_EXCHANGE = re.compile(
    r"(?:up\s*to|upto)\s*₹?\s*([\d,]{4,})\s*(?:off)?\s*on\s*exchange",
    re.I
)


def _find_product_container(title_div):
    """Walk up to the product card container (heuristic: container has a price div with class Nx9bqj)."""
    container = title_div
    for _ in range(8):
        container = container.parent
        if not container:
            break
        if container.find("div", class_=lambda c: c and "Nx9bqj" in c):
            return container
    return title_div.parent


# ===================== Scraper =====================

def scrape_page(soup):
    rows = []
    if soup is None:
        return rows

    # Anchor nodes: product titles
    for title_div in soup.select("div.KzDlHZ"):
        name = title_div.get_text(strip=True)
        container = _find_product_container(title_div)

        # ---------- Current price ----------
        current_price = np.nan
        price_div = container.find("div", class_=re.compile(r"\bNx9bqj\b")) if container else None
        if not price_div:
            price_div = soup.find("div", class_=re.compile(r"\bNx9bqj\b"))
        if price_div:
            current_price = _to_int_digits(price_div.get_text())

        # ---------- Exchange offer amount (robust) ----------
        exchange_offer_amount_max = np.nan
        offer_text = ""

        # Prefer the dedicated offer block with split fragments
        offer_block = container.find("div", class_=lambda c: c and "n5vj9c" in c) if container else None
        if offer_block:
            parts = [
                el.get_text(" ", strip=True)
                for el in offer_block.find_all(True, class_=lambda c: c and "yiggsN" in c)
            ]
            offer_text = " ".join(p for p in parts if p).strip()

        if offer_text:
            m = UPTO_EXCHANGE.search(offer_text)
            if m:
                exchange_offer_amount_max = int(m.group(1).replace(",", ""))

        if np.isnan(exchange_offer_amount_max):
            # Fallback: scan the entire card text, but still require the exact phrase
            card_txt = container.get_text(" ", strip=True) if container else ""
            m2 = UPTO_EXCHANGE.search(card_txt)
            if m2:
                exchange_offer_amount_max = int(m2.group(1).replace(",", ""))

        # ---------- Rating value ----------
        rdiv = container.find("div", class_="XQDdHH") if container else None
        rating_value = rdiv.get_text(strip=True).replace("★", "") if rdiv else np.nan

        # ---------- Ratings count (reviews_count intentionally omitted) ----------
        ratings_count = np.nan
        if container:
            rr = container.find(string=re.compile(r"Ratings"))
            if rr:
                ratings_count, _ = _parse_ratings_reviews(rr)

        # ---------- Specs under ul.G4BRas ----------
        spec_rom = spec_display = spec_camera = spec_processor = spec_warranty = np.nan
        if container:
            ul = container.select_one("ul.G4BRas")
            if ul:
                items = [li.get_text(strip=True) for li in ul.select('li.J\\+igdf') if li.get_text(strip=True)]
                def pick(pred):
                    for s in items:
                        if pred(s):
                            return s
                    return np.nan
                spec_rom       = pick(lambda s: "rom" in s.lower())
                spec_display   = pick(lambda s: "display" in s.lower() or "retina" in s.lower())
                spec_camera    = pick(lambda s: "camera" in s.lower())
                spec_processor = pick(lambda s: "processor" in s.lower() or "chip" in s.lower())
                spec_warranty  = pick(lambda s: "warranty" in s.lower())

        rows.append({
            "product_description": name,
            "current_price": current_price,
            "exchange_offer_amount_max": exchange_offer_amount_max,
            "rating_value": rating_value,
            "ratings_count": ratings_count,
            "spec_rom": spec_rom,
            "spec_display": spec_display,
            "spec_camera": spec_camera,
            "spec_processor": spec_processor,
            "spec_warranty": spec_warranty,
        })

    return rows


# ===================== Main =====================

if __name__ == "__main__":
    BASE_URL = (
        "https://www.flipkart.com/search?q=Iphone&otracker=search&otracker1=search"
        "&marketplace=FLIPKART&as-show=on&as=off"
    )
    TOTAL_PAGES = 26  

    driver = make_driver(headless=False)  # set True for headless mode
    all_rows = []
    try:
        for page in range(1, TOTAL_PAGES + 1):
            url = f"{BASE_URL}&page={page}"
            print(f"Scraping page {page}...")
            soup = get_page_soup(driver, url)
            if soup is None:
                print(f"  page {page}: timeout/no content → stopping.")
                break

            page_rows = scrape_page(soup)
            print(f"  found {len(page_rows)} products")
            if len(page_rows) == 0:
                print(f"  page {page}: 0 products → stopping.")
                break

            all_rows.extend(page_rows)
            time.sleep(0.8 + random.random() * 0.7)

        print("TOTAL PRODUCTS:", len(all_rows))
        if all_rows:
            df = pd.DataFrame(all_rows)
            print(df.head(5))
            df.to_csv("flipkart_Iphone.csv", index=False)
            print("Saved -> flipkart_phones.csv")
    finally:
        driver.quit()


Scraping page 1...
  found 24 products
TOTAL PRODUCTS: 24
                              product_description  current_price  \
0                 Apple iPhone 16 (White, 128 GB)          69900   
1                 Apple iPhone 16 (Black, 128 GB)          69900   
2             Apple iPhone 14 (Starlight, 128 GB)          54900   
3    Apple iPhone 16 Pro (White Titanium, 256 GB)         119900   
4  Apple iPhone 16 Pro (Natural Titanium, 256 GB)         119900   

   exchange_offer_amount_max rating_value  ratings_count    spec_rom  \
0                      47300          4.6         167138  128 GB ROM   
1                      47300          4.6         167138  128 GB ROM   
2                      41400          4.6         319483  128 GB ROM   
3                      47300          4.7          25436  256 GB ROM   
4                      47300          4.7          25436  256 GB ROM   

                                   spec_display  \
0  15.49 cm (6.1 inch) Super Retina XDR Display  

In [2]:
df

,product_description,current_price,exchange_offer_amount_max,rating_value,ratings_count,spec_rom,spec_display,spec_camera,spec_processor,spec_warranty
0,"Apple iPhone 16 (White, 128 GB)",69900,47300,4.6,167138,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
1,"Apple iPhone 16 (Black, 128 GB)",69900,47300,4.6,167138,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
2,"Apple iPhone 14 (Starlight, 128 GB)",54900,41400,4.6,319483,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,12MP + 12MP | 12MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
3,"Apple iPhone 16 Pro (White Titanium, 256 GB)",119900,47300,4.7,25436,256 GB ROM,16.0 cm (6.3 inch) Super Retina XDR Display,48MP + 48MP + 12MP | 12MP Front Camera,"A18 Pro Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
4,"Apple iPhone 16 Pro (Natural Titanium, 256 GB)",119900,47300,4.7,25436,256 GB ROM,16.0 cm (6.3 inch) Super Retina XDR Display,48MP + 48MP + 12MP | 12MP Front Camera,"A18 Pro Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
5,"Apple iPhone 16 (Ultramarine, 128 GB)",69900,47300,4.6,167138,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
6,"Apple iPhone 16 (Black, 256 GB)",79900,47300,4.6,167138,256 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
7,"Apple iPhone 16 (Teal, 256 GB)",79900,47300,4.6,167138,256 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
8,"Apple iPhone 14 (Blue, 128 GB)",54900,41400,4.6,319483,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,12MP + 12MP | 12MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
9,"Apple iPhone 16 (White, 256 GB)",79900,47300,4.6,167138,256 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...


### Samsung

In [3]:
# pip install selenium webdriver-manager beautifulsoup4 lxml numpy pandas

from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager

from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re, time, random


# ===================== Helpers =====================

def _to_int_digits(text):
    """Keep only digits and cast to int, else NaN."""
    if not text:
        return np.nan
    digits = re.sub(r"\D", "", text)
    return int(digits) if digits else np.nan


def _parse_ratings_reviews(text):
    """'25,319 Ratings & 1,097 Reviews' -> (25319, 1097)"""
    if not text:
        return np.nan, np.nan
    nums = [int(x.replace(",", "")) for x in re.findall(r"\d[\d,]*", text)]
    if len(nums) >= 2:
        return nums[0], nums[1]
    if len(nums) == 1:
        return nums[0], np.nan
    return np.nan, np.nan


def make_driver(headless=False):
    opts = Options()
    if headless:
        opts.add_argument("--headless")
    drv = webdriver.Firefox(
        service=FirefoxService(GeckoDriverManager().install()),
        options=opts
    )
    drv.set_page_load_timeout(60)
    return drv


def get_page_soup(driver, url, max_scrolls=3, wait_css="div.KzDlHZ"):
    """Load URL, close popup, wait for product titles, scroll a bit, return BeautifulSoup or None."""
    for attempt in range(2):
        try:
            driver.get(url)

            # Close login popup if visible
            try:
                WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button._2KpZ6l._2doB4z"))
                ).click()
            except Exception:
                pass

            WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, wait_css))
            )

            # Light scroll to load lazy content
            last_h = 0
            for _ in range(max_scrolls):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1.0)
                h = driver.execute_script("return document.body.scrollHeight;")
                if h == last_h:
                    break
                last_h = h

            return BeautifulSoup(driver.page_source, "lxml")

        except TimeoutException:
            if attempt == 0:
                time.sleep(2 + random.random() * 2)
                continue
            return None
    return None


# Regex for exchange phrase
UPTO_EXCHANGE = re.compile(
    r"(?:up\s*to|upto)\s*₹?\s*([\d,]{4,})\s*(?:off)?\s*on\s*exchange",
    re.I
)


def _find_product_container(title_div):
    """Walk up to the product card container (heuristic: container has a price div with class Nx9bqj)."""
    container = title_div
    for _ in range(8):
        container = container.parent
        if not container:
            break
        if container.find("div", class_=lambda c: c and "Nx9bqj" in c):
            return container
    return title_div.parent


# ===================== Scraper =====================

def scrape_page(soup):
    rows = []
    if soup is None:
        return rows

    # Anchor nodes: product titles
    for title_div in soup.select("div.KzDlHZ"):
        name = title_div.get_text(strip=True)
        container = _find_product_container(title_div)

        # ---------- Current price ----------
        current_price = np.nan
        price_div = container.find("div", class_=re.compile(r"\bNx9bqj\b")) if container else None
        if not price_div:
            price_div = soup.find("div", class_=re.compile(r"\bNx9bqj\b"))
        if price_div:
            current_price = _to_int_digits(price_div.get_text())

        # ---------- Exchange offer amount (robust) ----------
        exchange_offer_amount_max = np.nan
        offer_text = ""

        # Prefer the dedicated offer block with split fragments
        offer_block = container.find("div", class_=lambda c: c and "n5vj9c" in c) if container else None
        if offer_block:
            parts = [
                el.get_text(" ", strip=True)
                for el in offer_block.find_all(True, class_=lambda c: c and "yiggsN" in c)
            ]
            offer_text = " ".join(p for p in parts if p).strip()

        if offer_text:
            m = UPTO_EXCHANGE.search(offer_text)
            if m:
                exchange_offer_amount_max = int(m.group(1).replace(",", ""))

        if np.isnan(exchange_offer_amount_max):
            # Fallback: scan the entire card text, but still require the exact phrase
            card_txt = container.get_text(" ", strip=True) if container else ""
            m2 = UPTO_EXCHANGE.search(card_txt)
            if m2:
                exchange_offer_amount_max = int(m2.group(1).replace(",", ""))

        # ---------- Rating value ----------
        rdiv = container.find("div", class_="XQDdHH") if container else None
        rating_value = rdiv.get_text(strip=True).replace("★", "") if rdiv else np.nan

        # ---------- Ratings count (reviews_count intentionally omitted) ----------
        ratings_count = np.nan
        if container:
            rr = container.find(string=re.compile(r"Ratings"))
            if rr:
                ratings_count, _ = _parse_ratings_reviews(rr)

        # ---------- Specs under ul.G4BRas ----------
        spec_rom = spec_display = spec_camera = spec_processor = spec_warranty = np.nan
        if container:
            ul = container.select_one("ul.G4BRas")
            if ul:
                items = [li.get_text(strip=True) for li in ul.select('li.J\\+igdf') if li.get_text(strip=True)]
                def pick(pred):
                    for s in items:
                        if pred(s):
                            return s
                    return np.nan
                spec_rom       = pick(lambda s: "rom" in s.lower())
                spec_display   = pick(lambda s: "display" in s.lower() or "retina" in s.lower())
                spec_camera    = pick(lambda s: "camera" in s.lower())
                spec_processor = pick(lambda s: "processor" in s.lower() or "chip" in s.lower())
                spec_warranty  = pick(lambda s: "warranty" in s.lower())

        rows.append({
            "product_description": name,
            "current_price": current_price,
            "exchange_offer_amount_max": exchange_offer_amount_max,
            "rating_value": rating_value,
            "ratings_count": ratings_count,
            "spec_rom": spec_rom,
            "spec_display": spec_display,
            "spec_camera": spec_camera,
            "spec_processor": spec_processor,
            "spec_warranty": spec_warranty,
        })

    return rows


# ===================== Main (Samsung) =====================

if __name__ == "__main__":
    # Your provided URL (note: 'Samasung' is misspelled but used verbatim)
    BASE_URL = "https://www.flipkart.com/search?q=Samasung+phone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
    # If you want the corrected spelling, uncomment the line below for broader results:
    # BASE_URL = "https://www.flipkart.com/search?q=Samsung+phone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"

    TOTAL_PAGES = 26  # change as needed

    driver = make_driver(headless=False)  # set True for headless mode
    all_rows = []
    try:
        for page in range(1, TOTAL_PAGES + 1):
            url = f"{BASE_URL}&page={page}"
            print(f"Scraping page {page}...")
            soup = get_page_soup(driver, url)
            if soup is None:
                print(f"  page {page}: timeout/no content → stopping.")
                break

            page_rows = scrape_page(soup)
            print(f"  found {len(page_rows)} products")
            if len(page_rows) == 0:
                print(f"  page {page}: 0 products → stopping.")
                break

            all_rows.extend(page_rows)
            time.sleep(0.8 + random.random() * 0.7)

        print("TOTAL PRODUCTS:", len(all_rows))
        if all_rows:
            df = pd.DataFrame(all_rows)
            print(df.head(5))
            df.to_csv("flipkart_Samsung.csv", index=False)
            print("Saved -> flipkart_Samsung.csv")
    finally:
        driver.quit()


Scraping page 1...
  found 24 products
TOTAL PRODUCTS: 24
                               product_description  current_price  \
0       Samsung Galaxy F06 5G (Bahama Blue, 64 GB)           7499   
1     Samsung Galaxy A35 5G (Awesome Navy, 128 GB)          17999   
2      Samsung Galaxy F06 5G (Bahama Blue, 128 GB)           8999   
3  Samsung Galaxy A35 5G (Awesome Iceblue, 128 GB)          17999   
4              Samsung Galaxy F36 5G (Red, 128 GB)          13999   

   exchange_offer_amount_max rating_value  ratings_count  \
0                     5450.0          4.1          31917   
1                        NaN          4.4          52020   
2                     6700.0          4.2          15764   
3                        NaN          4.4          52020   
4                    10650.0          4.3           6314   

                                          spec_rom  \
0   4 GB RAM | 64 GB ROM | Expandable Upto 1500 GB   
1     8 GB RAM | 128 GB ROM | Expandable Upto 1 TB   
2  6

###  Datasets:

In [6]:
# Iphone dataset
dfi = pd.read_csv('Flipkart_Iphone.csv')
dfi.head()

,product_description,current_price,exchange_offer_amount_max,rating_value,ratings_count,spec_rom,spec_display,spec_camera,spec_processor,spec_warranty
0,"Apple iPhone 16 (White, 128 GB)",69900,47300.0,4.6,167138,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
1,"Apple iPhone 16 (Black, 128 GB)",69900,47300.0,4.6,167138,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,48MP + 12MP | 12MP Front Camera,"A18 Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
2,"Apple iPhone 16 Pro (White Titanium, 256 GB)",119900,47300.0,4.7,25436,256 GB ROM,16.0 cm (6.3 inch) Super Retina XDR Display,48MP + 48MP + 12MP | 12MP Front Camera,"A18 Pro Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
3,"Apple iPhone 14 (Starlight, 128 GB)",54900,41400.0,4.6,319483,128 GB ROM,15.49 cm (6.1 inch) Super Retina XDR Display,12MP + 12MP | 12MP Front Camera,"A15 Bionic Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...
4,"Apple iPhone 16 Pro (Natural Titanium, 256 GB)",119900,47300.0,4.7,25436,256 GB ROM,16.0 cm (6.3 inch) Super Retina XDR Display,48MP + 48MP + 12MP | 12MP Front Camera,"A18 Pro Chip, 6 Core Processor Processor",1 year warranty for phone and 1 year warranty ...


In [8]:
# Samsung dataset
dfs = pd.read_csv('Flipkart_Samsung.csv')
dfs.head()

,product_description,current_price,exchange_offer_amount_max,rating_value,ratings_count,spec_rom,spec_display,spec_camera,spec_processor,spec_warranty
0,"Samsung Galaxy F06 5G (Bahama Blue, 64 GB)",7499,5450.0,4.1,31917.0,4 GB RAM | 64 GB ROM | Expandable Upto 1500 GB,17.02 cm (6.7 inch) HD+ Display,50MP + 2MP | 8MP Front Camera,Dimensity 6300 Processor,1 Year Manufacturer Warranty for Device and 6 ...
1,"Samsung Galaxy F06 5G (Bahama Blue, 128 GB)",8999,6700.0,4.2,15764.0,6 GB RAM | 128 GB ROM | Expandable Upto 1500 GB,17.02 cm (6.7 inch) HD+ Display,50MP + 2MP | 8MP Front Camera,Dimensity 6300 Processor,1 Year Manufacturer Warranty for Device and 6 ...
2,"Samsung Galaxy A35 5G (Awesome Navy, 128 GB)",17999,NaN,4.4,52020.0,8 GB RAM | 128 GB ROM | Expandable Upto 1 TB,16.76 cm (6.6 inch) Full HD+ Display,50MP + 8MP + 5MP | 13MP Front Camera,Samsung Exynos 1380 Processor,1 Year Manufacturer Warranty for Device and 6 ...
3,"Samsung Galaxy A35 5G (Awesome Iceblue, 128 GB)",17999,NaN,4.4,52020.0,8 GB RAM | 128 GB ROM | Expandable Upto 1 TB,16.76 cm (6.6 inch) Full HD+ Display,50MP + 8MP + 5MP | 13MP Front Camera,Samsung Exynos 1380 Processor,1 Year Manufacturer Warranty for Device and 6 ...
4,"Samsung Galaxy F36 5G (Red, 128 GB)",13999,10650.0,4.3,6314.0,6 GB RAM | 128 GB ROM | Expandable Upto 2 TB,17.02 cm (6.7 inch) Full HD+ Display,50MP + 8MP + 2MP | 13MP Front Camera,Samsung Exynos 1380 Processor,1 Year Manufacturer Warranty for Device and 6 ...
